In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
import json
import logging
from  pyspark.sql.functions import input_file_name

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars jars/aws-java-sdk-1.7.4.jar,jars/delta-core_2.12-0.7.0.jar,jars/hadoop-aws-2.7.3.jar pyspark-shell'

In [3]:
conf = (
SparkConf()
    .set("spark.hadoop.fs.s3a.endpoint", "localhost:9000")
    .set("spark.hadoop.fs.s3a.access.key", "9TPEM7DXMUVP5WUWA21I")
    .set("spark.hadoop.fs.s3a.secret.key", "UPiPw9nhsGZ67R1smKF3ucClvMn9d1rwb9kUpNrY")
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.fast.upload", True)
    .set("spark.hadoop.fs.s3a.connection.maximum", 500)
    .set("spark.hadoop.fs.s3a.connection.timeout", 600000)
    .set("spark.hadoop.fs.s3a.multipart.size",104857600)
    .set("spark.hadoop.fs.s3a.multipart.threshold", 2097152000)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

In [4]:
sc = SparkContext(conf=conf).getOrCreate()

21/11/14 10:36:43 WARN Utils: Your hostname, DESKTOP-FEECCSC resolves to a loopback address: 127.0.1.1; using 172.27.184.151 instead (on interface eth0)
21/11/14 10:36:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/14 10:36:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = SparkSession \
            .builder \
            .appName("sz-stock-analytics") \
            .getOrCreate()

In [6]:
spark.sparkContext.setLogLevel("INFO")
logging.basicConfig(level=logging.INFO)

In [7]:
df = spark.read.csv("s3a://stock-market/lz/*.csv",header=True)

Py4JJavaError: An error occurred while calling o56.csv.
: java.lang.IllegalArgumentException
	at java.util.concurrent.ThreadPoolExecutor.<init>(ThreadPoolExecutor.java:1314)
	at java.util.concurrent.ThreadPoolExecutor.<init>(ThreadPoolExecutor.java:1237)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:280)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:377)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameRea

In [40]:
df_statistics = spark.sql("""
WITH raw_data AS (
SELECT
    CAST(timestamp AS TIMESTAMP) AS period,
    CAST(CAST(timestamp AS TIMESTAMP) AS DATE) AS day,
    FROM_UNIXTIME(UNIX_TIMESTAMP(timestamp),'H') AS hour,
    open,
    high,
    low,
    close,
    volume,
    REGEXP_EXTRACT(filename,"(?<=\/)[A-Z]+(?=-)",0) AS company
FROM 
    stocks
),
concat_period AS (
SELECT
    CONCAT(day,'|',hour) AS period,
    company,
    open,
    high,
    low,
    close,
    volume
FROM raw_data
)
SELECT
    company,
    period,
    AVG(open) AS mean_open,
    MAX(high) AS max_high,
    MIN(low) AS min_low,
    AVG(close) AS mean_close,
    AVG(volume) AS mean_volume
FROM 
    concat_period
GROUP BY
    company,
    period
ORDER BY 
    company,
    period
""")


In [ ]:
df_statistics.write.format("delta").saveAsTable("s3a://stock-market/sz/stock-statistics")